Załadowanie bibliotek i utworzenie obiektu Login

In [ ]:
#!import ./../load.ipynb

Utworzenie statycznego obiektu Meter dla rozszerzenia. Należy pamiętać o włączeniu go w appsettings.json (Telemetry:MetricsSources).




In [ ]:
using System.Diagnostics.Metrics;

static Meter MyMeter = new Meter("BestSonetaAddon");

Utworzenie statycznego obiektu Counter zliczającego przetworzone towary.

In [ ]:
static Counter<int> ProductsCounter = MyMeter.CreateCounter<int>("ProcessedProducts", unit: "pcs");

Utworzenie statycznego obiektu Gauge, czyli metryki bazującej na delegacie obliczającej wartość za każdym razem, kiedy jest odpytywana.

In [ ]:
static int lastWarningCount = 0;
static ObservableGauge<int> WarningGauge = MyMeter.CreateObservableGauge<int>("WarningsFromLastRun", () => lastWarningCount);

Utworzenie statycznego obiektu Histogram zbierającego czas przetwarzania towarów.

In [ ]:
static Histogram<long> ProductsProcessingTimeHistogram = MyMeter.CreateHistogram<long>("ProductsProcessingTime", unit: "ms");

Użycie metryki ProcessedProducts oraz histogramu ProductsProcessingTime w algorytmie przetwarzającym towary


In [ ]:
using System.Diagnostics;
using Soneta.Towary;


lastWarningCount = 0;
using(var session = Login.CreateSession("Pobieranie listy towarów"))
{
  var stopWatch = new Stopwatch();
  stopWatch.Start();
  int validCount = 0;
  foreach(var towar in session.GetTowary().Towary.WgNazwy)
  {    
    ProductsCounter.Add(1);
    if (string.IsNullOrWhiteSpace(towar.EAN))
      lastWarningCount++;
    else
      validCount++;
    WriteLine(towar.Nazwa);
  }  
  stopWatch.Stop();
  ProductsProcessingTimeHistogram.Record(stopWatch.ElapsedMilliseconds);
}

Wyczyszczenie obiektu Login

In [ ]:
Login?.Dispose();